# Post Trade Analysis

Note: Please drag the SQLite file to the folder where you ran this notebook.

## All Buy-Sell Pairs + Opportunity
Check details of each trade decision.

In [ ]:
import sqlite3

import pandas as pd

# Database connection
conn = sqlite3.connect("crypto.sqlite")

# Read candlesticks from database
df = pd.read_sql("select * from trade_result", con=conn)
df["profit"] = df["sell_trades.0.price"] * df["sell_trades.0.quantity"] - df["buy_trades.0.price"] * df["buy_trades.0.quantity"]
df_result = df[[
    "opportunity.bull_flag_pattern.start",
    "opportunity.score",
    "opportunity.risk_reward_ratio",
    "opportunity.profit_price",
    "opportunity.expected_trade_price",
    "opportunity.stop_loss_price",
    "buy_trades.0.price",
    "sell_trades.0.price",
    "profit"
]]
df_result

 ## Win Rate

In [ ]:
result_df = pd.read_sql("select * from trade_result", con=conn)
result_df

In [ ]:
win_rate = len(df[df["profit"] > 0]) / len(df) * 100
print(f"Win Rate: {win_rate:.2f}%")
pnl = sum(df["profit"])
print(f"PnL: {pnl:.2f}")

## Score vs. PnL
Check if our score system is good or not.

In [ ]:
df.plot(
    kind="scatter",
    x="opportunity.score",
    y="profit",
)

# Plot Candlesticks and Trades


In [ ]:
# Enable autoreload extension
%load_ext autoreload
%autoreload 2

In [ ]:
from analysis.scripts.post_trade_plot import PostTradePlot

tool = PostTradePlot("crypto.sqlite")
tool.draw()

In [ ]:
tool.load_candlesticks("calculated_candlestick_feed")